<a href="https://colab.research.google.com/github/Arina1308/SAS_Text_Project/blob/main/SAS_emitents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2
!pip install natasha
import pandas as pd
import numpy as np
from itertools import compress
import re
from tqdm import tqdm
import warnings
import nltk
warnings.filterwarnings('ignore')
from nltk.tokenize import sent_tokenize
import pymorphy2
nltk.download('punkt')
from natasha import MorphVocab, NamesExtractor

py2 = pymorphy2.MorphAnalyzer()
morph_vocab = MorphVocab()
extractor = NamesExtractor(morph_vocab)

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 8.2MB 13.4MB/s 
     |████████████████████████████████| 34.4MB 124kB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26120 sha256=e722da128f6b773b29dee249f6b175322c46317bd2c34dd3177f1f98e0443c98
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
! git clone https://github.com/Anna-Pon/SASemitents

Cloning into 'SASemitents'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 23 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (23/23), done.


# Автоматизация работы с документами: извлечение сущностей и фактов из сообщений о раскрытии


Задача:
Собрать не менее 300 решений общих собраний участников (акционеров) с сайта e-disclosure.ru в формате .html страниц.
Разработать инструмент по парсингу html страниц и извлечению из них текста решения.

Разработать инструмент по извлечению из данных решений следующей информации:

Полное и сокращенное наименование эмитента

Адрес, ИНН, ОГРН эмитента

Дата и форма собрания

Наименование и ИНН утвержденного аудитора + тип отчетности, которую ему поручено проверять (при наличии)

Утвержденный состав совета директоров (при наличии)

Поднимался ли на собрании вопрос о выплате дивидендов и если да, то какое решение принято (3 варианта: “принято решение выплатить дивиденды”, “принято решение не выплачивать дивиденды”, “вопрос не поднимался”)

# Сбор данных
Данный раздел можно не запускать, табличка с собранными текстами отдельно подгружается далее

In [ ]:
#website_content = open('website.txt').read()
website_content = open('/content/SASemitents/website.txt').read()

In [ ]:
from bs4 import BeautifulSoup
import requests

# распарсили страничку в дерево 
tree = BeautifulSoup(website_content, 'html.parser')
table_with_links = tree.find_all('table', {'class': 'live noBorderTbl'})
a_list = table_with_links[0].find_all('a')

names_inds = list(np.arange(0, 1060, 2))
links_inds = list(np.arange(1, 1060, 2))

mapping_names = map(a_list. __getitem__, names_inds)
mapping_links = map(a_list. __getitem__, links_inds)
names  = list(mapping_names)
links = list(mapping_links)

parsed_table = pd.DataFrame({'company_name': names, 'link': links})

parsed_table['company_name'] = parsed_table['company_name'].apply(lambda x: x.text)
parsed_table['link'] = parsed_table['link'].apply(lambda x: str(x['href']))

In [ ]:
parsed_table

,company_name,link
0,"ПАО ""Бест Эффортс Банк""",https://www.e-disclosure.ru/portal/event.aspx?...
1,"ЗАО Агрофирма ""Нива""",https://www.e-disclosure.ru/portal/event.aspx?...
2,"ОАО ""ДОРСТРОЙ""",https://www.e-disclosure.ru/portal/event.aspx?...
3,ООО КСН «Структурные инвестиции 1»,https://www.e-disclosure.ru/portal/event.aspx?...
4,"ОАО ""КАНАТ""",https://www.e-disclosure.ru/portal/event.aspx?...
...,...,...
525,"АО ""ЦГРМ ""ГЕНЕТИКО""",https://www.e-disclosure.ru/portal/event.aspx?...
526,"ПАО ""ДИОД""",https://www.e-disclosure.ru/portal/event.aspx?...
527,"ОАО ""ТСК""",https://www.e-disclosure.ru/portal/event.aspx?...
528,"АО ""Смоленскмебель""",https://www.e-disclosure.ru/portal/event.aspx?...


In [ ]:
def get_text(link):
  response = requests.get(link).content
  tree = BeautifulSoup(response, 'html.parser')
  return tree.find_all('div', {'id': 'cont_wrap'})[0].find_all('div')[1]

In [ ]:
parsed_table['content'] = parsed_table['link'].apply(lambda x: get_text(x))

In [ ]:
parsed_table

In [ ]:
# parsed_table.to_excel('SAS project data.xlsx', index=False)

# Вычленение информации из собранных данных

In [ ]:
parsed_table = pd.read_excel('/content/SASemitents/SAS_project_data.xlsx')

- #### Полное название, Сокращенное название, ОГРН, ИНН, Место нахождения эмитента

In [ ]:
def get_name(x):
  x = str(x)
  ind = x.lower().find('полное фирменное наименование')
  full_name = x[ind:x[ind:].find('<br/>')+ind - 1].split(')')[-1].strip('\t').strip('\r').strip('\xa0').strip(':').strip('Полное фирменное наименование эмитент').strip(' : ').strip('\t')
  try:
    if full_name[0] == 'у':
      full_name = 'П' + full_name
  except:
    full_name = full_name
  return full_name 

In [ ]:
parsed_table['Полное название'] = parsed_table['content'].apply(get_name)

In [ ]:
parsed_table[parsed_table['Полное название'] == '']

In [ ]:
parsed_table['Сокращенное название'] = parsed_table['company_name']

In [ ]:
def get_full_name(text, lookup):
  for br in str(text).split('<br/>'):
    if lookup in br:
      return br

def points(text, lookup):
  if ':' in text:
    return text[text.find(':')+1:]
  elif '\t' in text:
    return text[text.find('\t')+1:]
  else:
    return text[text.find(lookup):]

# parsed_table['Полное название'] = parsed_table['content'].apply(lambda x: points(get_full_name(x, 'Полное фирменное наименование эмитента'), 'Полное фирменное наименование эмитента').strip('\r'))

In [ ]:
tolookfor='ИНН'
parsed_table['ИНН'] = parsed_table.content.apply(lambda x: points(get_full_name(x, tolookfor), tolookfor).strip(' ').strip('\r').strip('\t').strip('\xa0'))
parsed_table['ИНН'] = parsed_table['ИНН'].apply(lambda x: x.replace('\t', ' ').split(' ')[-1])

In [ ]:
tolookfor='Место нахождения эмитента'
parsed_table['Адрес'] = parsed_table.content.apply(lambda x: points(get_full_name(x, tolookfor), tolookfor).strip(' ').strip('\r').strip('\t').strip('\xa0'))

In [ ]:
'''
tolookfor='ОГРН эмитента'
parsed_table['ОГРН'] = parsed_table.content.apply(lambda x: points(get_full_name(x, tolookfor), tolookfor).strip(' ').strip('\r').strip('\t').strip('\xa0'))
parsed_table['ОГРН'] = parsed_table['ОГРН'].apply(lambda x: x.replace('\t', ' ').split(' ')[-1]).astype(int)
'''

In [ ]:
def get_ogrn(x):
  x = str(x)
  ind = x.find('ОГРН эмитента')
  ogrn = x[ind:x[ind:].find('<br/>')+ind].split('ОГРН эмитента')[-1].strip('\t').strip('\r').strip('\xa0').strip(':').strip(' ').strip(';')
  try:
    ogrn = int(ogrn)
  except:
    ogrn = x[ind:x[ind:].replace('<br/>', 'XXXXX', 1).find('<br/>')+ind].split('ОГРН эмитента')[-1].strip('.<br/>').strip('\t').strip('\r').strip('\xa0').strip(':').strip(' ')
    try:
      ogrn = int(ogrn)
    except:
      ogrn = x[ind:x[ind:].find('<br/>')+ind].split('–')[-1].strip('\t').strip('\r').strip('\xa0').strip('-').strip(' ')
  return ogrn

In [ ]:
parsed_table['ОГРН'] = parsed_table.content.apply(get_ogrn)

- #### Дата проведения собрания

In [ ]:
def find_date(x):
  x = x.lower().replace('июля', '07').replace('августа', '08').replace('сентября', '09')
  for i in x.split('<br/>'):
    a = i.split('\t')
    for m in a:
      if 'дата' in m:

        try:
          date = pd.to_datetime(i.strip('г.').strip('года').split()[-1], dayfirst=True)

          if date.month < 7: # если выполняется, значит, увидел только год, либо не та дата
            1/0 # просто ломаем код
          if date.year < 2020: # если выполняется, значит, не та дата
            break

          return date

        except:
          i = i.replace('г.', ' г.') # вставим пробел там, где его не хватает, чтобы сплитить спокойно
          date = '.'.join(i.split()[-4:-1]).replace('«', '').replace('»', '') # собираем составную дату

          try:
            date = pd.to_datetime(date, dayfirst=True) # проверяем, что datetime работает

            if date.month == 1:
              1/0
            if date.year < 2020:
              break

            return date

          except:
            continue # если не нашел дату, идем дальше по циклу


In [ ]:
parsed_table['Дата собрания'] = parsed_table.content.apply(find_date)

In [ ]:
parsed_table[parsed_table['Дата собрания'].isna()]
# 4 не достались, в них либо особенность - дата написана посреди предложения, либо там переход на новый абзац, 
# вообще можно добить, но вроде некритично

In [ ]:
parsed_table[parsed_table['Дата собрания'] < '2020-07-01'] # всё корректно досталось

- #### Форма собрания

In [ ]:
def find_form(x):
  x = str(x).lower()
  ind = -1 
  look = ['форма проведения', 'форма поведения', 'общего собрания путем', 'общего собрания акционеров путем',
          'форма и вид проведения', 'вид и форма общего собрания', 'форма собрания']
  i=0
  while ind == -1:
    ind = x.find(look[i])
    i+=1
    if i >= len(look):
      break


  if (x.find('место проведения') != -1) or\
  (x.find('дата, место, время проведения') != -1) or\
  (x.find('дата, ме-сто, время проведения') != -1) or\
  (x.find('место (адрес) проведения') != -1) or\
  (x.find('личное присутствие') != -1) or\
  (x.find('будет проводиться по адресу') != -1):
    return 'очное'
  elif ind == -1:
    return np.nan
  else:
    if (x[ind:].find('заочн') != -1) or (x[ind:].find('Заочн') != -1):
      return 'заочное'
    elif (x[ind:].find('совместное присутствие') != -1) or (x[ind:].find('совместного присутствия') != -1):
      return 'очное'
    else:
      return np.nan


In [ ]:
parsed_table['Форма собрания'] = parsed_table.content.apply(find_form)

In [ ]:
parsed_table['Форма собрания'].value_counts()

- #### Дивиденды

In [ ]:
def div_agenda(x): # ищем, есть ли упоминание дивидендов в решении
  x = str(x.lower())

  if ('дивид' in x) or\
  ('распределение чистой прибыли' in x) or\
  ('о распределении чистой прибыли 'in x) or\
  ('распределение прибыли' in x):
    return np.nan
  else:
    return 'вопрос не поднимался'


def div_res(x): # проверяем решение по дивам
  x = str(x).lower()
  ind1 = x.find('не распред')
  if ind1 == -1:
    ind1 = x.find('не выплач')
  if ind1 == -1:
    ind1 = x.find('невыпла')
  if ind1 == -1:
    ind1 = x.find('не начисл')
  if ind1 == -1:
    ind1 = x.find('не производ')

# смотрим, есть ли связь между отрицательным сказуемым и дивидендами, для этого задаем интервал, внутри которого должны находиться оба слова
  ind2 = x[ind1- 400:ind1+400].find('дивид')
  ind3 = x[ind1- 400:ind1+400].find('прибыль')
  ind4 = x[ind1- 400:ind1+400].find('убыт')

  if (ind1 != -1) and (ind2 != -1 or ind3 != -1 or ind4 != -1):
    return 'принято решение не выплачивать дивиденды'
  else:
    return 'принято решение выплатить дивиденды'

  # есть неточности, но в целом (примерно 90% достает корректно) очень даже неплохо работает

In [ ]:
parsed_table['Дивиденды'] = parsed_table.content.apply(div_agenda)
parsed_table.loc[parsed_table['Дивиденды'].isna(), 'Дивиденды'] = parsed_table[parsed_table['Дивиденды'].isna()].content.apply(div_res)

In [ ]:
parsed_table['Дивиденды'].value_counts()

- #### Наименование и ИНН утвержденного аудитора 

In [ ]:
found = parsed_table
data = parsed_table

In [ ]:
indices = np.array([])
for i in range(len(data)):
    indices = np.append(indices, data['content'][i].lower().find('удитор'))

In [ ]:
found['Аудитор'] = ''
found['Аудитор'][np.where(indices < 0)[0]] = 'Аудитора нет'

In [ ]:
signs_comp = ['ООО', 'ЗАО', 'ОАО', 'ПАО', 'Общество', 'АО'] # знаки для поиска компании
punct = '#$%&\()*,/:;<=>?@[\\]^_`{|}~' # белый шум
# данная функция убирает из текста возможные наименования фирмы, которая нанимает аудитора
def remove_comp_name(x, string):
    comp = found['company_name'][x]
    n = comp.count('"')
    if n > 2:
        # если кавычек больше двух, то последние две заменим на другие
        string = string.replace(comp.replace('"', '«', n - 1).replace('"', '»', 1), '')
    # убираем название компании и все производные от него
    string = string.replace(comp, '') 
    string = ' '.join(string.replace(comp.replace('№', ''), '').split())
    string = ' '.join(string.replace('№', '').replace(comp, '').split())
    string = string.replace(comp.replace('"', '«', 1).replace('"', '»', 1), '')
    string = string.replace(comp.replace('«','"').replace('»','"'), '')
    for elem in signs_comp:
        comp = comp.replace(elem, '')
    # убиарем пунктуацию
    for i in range(len(punct)):
        comp = comp.replace(punct[i], '')
    comp = comp.strip()
    # убираем еще варианты того, как можно переделать название компании
    for elem in signs_comp:
        string = string.replace(elem + ' ' + comp, '')
    try:
        string = string.replace(found['Полное название'][x].split('"')[-2], '')
    except:
        try:
            string = string.replace('«' + found['Полное название'][x].split('«')[-1], '')
        except:
            pass
    # здесь учитываем случай, когда в названии компании делается ошибка (например, вместо русской с пишется английская c)
    s = string.split()
    n = len(comp.split())
    for x in range(0, len(s) - n):
        s2 = ' '.join(s[x:x + n])
        # если расстояние между словами меньше 5 (то есть меньше 5 букв различается), то это название компании
        if nltk.edit_distance(comp, s2) < 5:
            del s[x: x + n]
            break
    string = ' '.join(s)
    # убираем фирму реестр
    return string.replace('ООО "Реестр-РН"', '').strip()

# работаем с теми текстами, где есть вхождения:
for i in np.where(indices != -1)[0]:
    # находим все куски текста, где есть данные вхождения
    t = [m.start() > [m.start() for m in re.finditer('удитор', data['content'][i].lower())][-1]
                for m in re.finditer('<br/>', data['content'][i].lower())]
    # убираем ту часть текста, которая до первого вхождения, и ту, которая после последнего вхождения (с некоторым интервалом)
    left = [m.start() for m in re.finditer('<br/>', data['content'][i].lower())][list(compress(range(len(t)), t))[0] - 1]
    right = [m.start() for m in re.finditer('<br/>', data['content'][i].lower())][list(compress(range(len(t)), t))[0] + 1]
    # убираем разметку <br\>
    string = data['content'][i][left + 5:right - 1].replace('<br/>', '').replace('<br/', '')
    # знаки для поиска аудитора
    signs = ['ООО', 'ЗАО', 'ОАО', 'Акционерное', 'акционерное', 'Общество', 'общество', 'АО', "АКЦИОНЕРНОЕ"]
    # удаляем название компании
    string = remove_comp_name(i, string)
    k = 0
    string = string[:600]
    # ищем последнее вхождение знака
    for elem in signs:
        c = string.rfind(elem)
        if c > k:
            k = c
    string = string[k:]
    # ищем кавычки, так как они находят фирмы
    if len(string[:string.find('»') + 1]) > 0:
        found['Аудитор'][i] = string[:string.find('»') + 1]
    elif len(string[:string.replace('"', ',', 2).find('"') + 1]) > 0 and string.replace('"', ',', 2).find('"')  - string.replace('"', ',', 1).find('"')  < 25:
        found['Аудитор'][i] = string[:string.replace('"', ',',2 ).find('"') + 1]
    elif len(string[:string.replace('"', ',', 1).find('"') + 1]) > 0:
        found['Аудитор'][i] = string[:string.replace('"', ',', 1).find('"') + 1]
    elif len(string[string.find('«'):string.find('»') + 1]) > 0:
        found['Аудитор'][i] = string[string.find('«'):string.find('»') + 1]
    elif len(string[string.find('«'):string.find('"') + 1]) > 0:
        found['Аудитор'][i] = string[string.find('«') - 4:string.find('"') + 1]
    else:
        found['Аудитор'][i] = string
        k = 0
    # если предыдущие правила не сработали, выводим доделать
    if k == 0 or len(found['Аудитор'][i].split()) == 1:
        found['Аудитор'][i] = 'Доделать'

# второй цикл для тех текстов, где нужно доделать
for i in np.where(found['Аудитор'] == 'Доделать')[0]:
    # если вхождение ровно одно, то выводим, что аудитора нет, так как вхождений должно быть минимум 2 
    # (упоминание в повестке дня и в отдельном рассмотрении)
    if len([m.start() for m in re.finditer('удитор', data['content'][i].lower())]) == 1:
        found['Аудитор'][i] = 'Аудитора нет'
    # ищем все вхождения по текстам
    t = [m.start() > [m.start() for m in re.finditer('удитор', data['content'][i].lower())][-1]
            for m in re.finditer('<br/>', data['content'][i].lower())]
    # в этот раз берем большее окно 
    left = [m.start() for m in re.finditer('<br/>', data['content'][i].lower())][list(compress(range(len(t)), t))[0] - 3]
    right = [m.start() for m in re.finditer('<br/>', data['content'][i].lower())][list(compress(range(len(t)), t))[0] + 3]
    # удаляем разметку
    string = data['content'][i][left + 5:right - 1].replace('<br/>', '').replace('<br/', '')
    # выписываем знаки и удаляем название компании
    signs = ['ООО', 'ЗАО', 'ОАО', 'Акционерное', 'акционерное', 'Общество', 'общество', 'АО', "АКЦИОНЕРНОЕ"]
    string = remove_comp_name(i, string)
    # удаляем особые кейсы, когда фирма не раскрывает сведения (как ПАО Звезда):
    if string.lower().find('не раскрываются') != -1:
        found['Аудитор'][i] = 'Сведения не раскрываются'
    # иначе ищем те куски текста, где есть выражение, рядом с которым обычно находятся названия фирм-аудиторов
    else:
        # убираем за против воздержался, так как они обычно в кавычках
        string = string.lower().replace('«за»', '').replace('«против»', '').replace('«воздержался»', '')
        # ищем фразу утвердить аудитором
        string = string[string.lower().find('утвердить аудитором'):]
        k = 0
        # в данной подстроке ищем наки
        for elem in signs:
            c = string.rfind(elem)
            if c > k:
                k = c
                break
        string = string[k:]
        # далее поиск аудитора по кавычкам
        if len(string[:string.find('»') + 1]) > 0:
            found['Аудитор'][i] = string[:string.find('»') + 1]
        elif len(string[:string.replace('"', ',', 2).find('"') + 1]) > 0 and string.replace('"', ',', 2).find('"')  - string.replace('"', ',', 1).find('"')  < 25:
            found['Аудитор'][i] = string[:string.replace('"', ',',2 ).find('"') + 1]
        elif len(string[:string.replace('"', ',', 1).find('"') + 1]) > 0:
            found['Аудитор'][i] = string[:string.replace('"', ',', 1).find('"') + 1]
        elif len(string[string.find('«'):string.find('»') + 1]) > 0:
            found['Аудитор'][i] = string[string.find('«'):string.find('»') + 1]
        elif len(string[string.find('«'):string.find('"') + 1]) > 0:
            found['Аудитор'][i] = string[string.find('«') - 4:string.find('"') + 1]
        # если не сработали кавычки, ищем по пунктуации
        elif len(string[:string.find(',') + 1]) > 0:
            found['Аудитор'][i] = string[:string.find(',')]
        else:
            found['Аудитор'][i] = string
            k = 0
        # если ничего не нашли, то выводим доделать
        if k == 0 or len(found['Аудитор'][i].split()) == 1:
            found['Аудитор'][i] = 'Доделать'
# обработка, чтобы все названия аудиторов выводились с большой буквы
found['Аудитор'] = found['Аудитор'].apply(lambda x: re.sub('([а-яА-Я])', lambda x: x.groups()[0].upper(), x, 1).strip())

In [ ]:
print('Нужно доделать:', len(np.where(found['Аудитор'] == 'Доделать')[0]))
print('Всего текстов:', len(found))

In [ ]:
found['ИНН Аудитора'] = ' '
for i in range(len(found)):
    # ищем вхождения подстроки 'удитор'
    try:
        t = [m.start() > [m.start() for m in re.finditer('удитор', data['content'][i].lower())][-1]
             for m in re.finditer('<br/>', data['content'][i].lower())]
    except:
        t = -29
    if t != -29:
        # если такие вхождения есть, то ищем куски текста с аудиторами
        left = [m.start() for m in re.finditer('<br/>', data['content'][i].lower())][list(compress(range(len(t)), t))[0] - 1]
        right = [m.start() for m in re.finditer('<br/>', data['content'][i].lower())][list(compress(range(len(t)), t))[0] + 1]
        string = data['content'][i][left + 5:right - 1].replace('<br/>', '').replace('<br/', '')
        # с помощью регулярного выражения ищем 10-значное число, рядом с которым есть слово ИНН
        try:
            found['ИНН Аудитора'][i] = int(re.findall(r'\b\d{10}\b', string)[0]) * ('ИНН' in string)
        except:
            pass

- #### Совет Директоров

In [ ]:
data = parsed_table.copy()

In [ ]:
def get_board(content):
  ind = 0
  text = ' '
  i = 0
  # если ничего не упоминается про совет директоров, то выбора тем более не было
  if content.lower().find('совет директоров') == -1 and content.lower().find('совета директоров') == -1:
      if content.lower().count('директор') <= 1 or (content.lower().count('совет директор') == 0 and content.lower().count('совета директор') == 0):
        text = 'Вопрос не поднимался'
  # иначе циклом ищем тот кусок текста, где есть совет директоров, но уже нет ревизионной комиссии
  else:
      while len(text) < 150:
        x = content[ind + 10:]
        ind = x.lower().find('совет директоров') + ind
        if ind == -1:
            ind = x.lower().find('совета директоров') + ind
        if ind != -1 + ind:
          ind1 = x[ind:].lower().find('ревизион') 
          if ind1 != -1:
            text = x[ind:ind1+ind]
          else:
            ind1 = x[ind:].lower().find('аудитор')
            if ind1 != -1:                                 
              text = x[ind:ind1+ind] 
            else:
              text = x[ind:] + ' '*100
        elif ind == -1 and content.lower().count('директор') <= 1:
              text = 'Вопрос не поднимался'
        
        # Это чтобы не допустить вечный цикл
        else:
          text = ' '*150
        
        i +=1
        if i >= 10:
          text = ' '*150
      
      # чтобы легче было увидеть ненайденные 
      if text == ' '*150:
        text = np.nan

      # проверка, чтобы точно не обрабатывать потом куски, где нет совета директоров
      if text == np.nan:
        ind = 0
        ind = content.lower().find('совет директоров') + ind
        if ind == -1:
            ind = content.lower().find('совета директоров') + ind
        if ind == -1 and content.lower().count('директор') <= 1:
            text = 'Вопрос не поднимался'
        #if type(text) != 'str':
        #    text = 'Доделать'
      # предобработка
      text = str(text)
      text = text.replace('<br/>', ' ')
      text = text.replace('\t', ' ')
      text = text.replace('\t0', ' ')
      text = text.replace('\xa0', ' ')
      text = ' '.join(re.sub( r"([А-Я])", r" \1", text).split())
      replace_list = ['Итог', 'Решение', 'Вопрос', 'Фамилия', 'Число', 'Дата', 'За', 'Против', 'Воздержался', 'Общество', 'Банк', 'Предложение', 'Результат', 'Формулировка', 'Идентификационный', 'Договор', 'Бывший', 'Бест', 'Предоставить', 'Избравший', 'Уполномоченное']
      text = re.sub(r'|'.join(map(re.escape, replace_list)), ' ', text)
      if text != 'Вопрос не поднимался' and text != np.nan:
          text = sent_tokenize(text)
          text = np.unique(text)

      if np.isscalar(text) != True:
          text = ' '.join(text)
          matches = extractor(text)
          facts = [_.fact for _ in matches if (_.fact.first != None and _.fact.last != None)]
          dir = []
          for fact in facts:
            try:
              last = py2.parse(fact.last)[0].inflect({'sing', 'nomn'}).word.capitalize()
            except:
              last = fact.last
            try:
              first = py2.parse(fact.first)[0].inflect({'sing', 'nomn'}).word.capitalize()
            except:
              first = fact.first
            try:
              middle = py2.parse(fact.middle)[0].inflect({'sing', 'nomn'}).word.capitalize()
            except:
              try:
                middle = fact.middle
              except:
                middle = str()
            person = last + ' ' + first 
            if middle != None:
              person = person + ' ' + middle
            dir.append(person)
      text = dir
      if len(text) <= 1:
        text = 'Вопрос не поднимался'
      if np.isscalar(text) != True:
        text = np.unique(text)
  # возвращаем список людей, которые были избраны в совет директоров
  return text

In [ ]:
# вытащим СД
data['Совет директоров'] = data.content.apply(get_board)

# Так как у нас нет разметки, а пустых ячеек тоже нет, то сложно судить о качестве данной модели, но в случайной выборке из 10 наблюдений все списки оказались верными.

In [ ]:
# в презу - вопрос не поднимался + 36, 49, 196, 161, 54, 284, 357, 242, 307, 157, 408, 259, 251, 511, 514, 456
#data.drop(columns = ['content'], inplace = True)
i = 197
data[i:i + 1]

### Что еще нужно собрать:

~~Дата собрания~~

~~Форма собрания~~

~~Наименование и ИНН утвержденного аудитора~~

~~Утвержденный состав совета директоров (при наличии)~~

~~Поднимался ли на собрании вопрос о выплате дивидендов и если да, то какое решение принято (3 варианта: “принято решение выплатить дивиденды”, “принято решение не выплачивать дивиденды”, “вопрос не поднимался”)~~

In [ ]:
# parsed_table.to_excel('parsed and found.xlsx', index=False)
# partial_res = pd.read_excel('/content/SASemitents/parsed_and_found.xlsx')

In [ ]:
data.drop(columns = ['content'], inplace = True)
data.head()

In [ ]:
data.to_excel("prs.xlsx")